In [1]:
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder

from joblib import dump, load

import numpy as np

In [212]:
#loading saved model
regressor = load('20220818_randomforest_model.joblib')

In [229]:
xl1 = pd.read_csv('data_cleaned/modelSource.csv', index_col=0)


In [230]:
# Account for the way WRS are set in slot, left, and right
xl1['posRank'] = xl1['posRank'].map({
    'QB1':'QB1', 
    'QB2':'QB2', 
    'QB3':'QB3', 
    'QB4':'QB4',
    'QB5':'QB5', 
    'RB1':'RB1', 
    'RB2':'RB2', 
    'RB3':'RB3', 
    'RB4':'RB4', 
    'RB5':'RB5',
    'RB6':'RB5', 
    'RB7':'RB5', 
    'WR1': 'WR1', 
    'WR2': 'WR1', 
    'WR3': 'WR1', 
    'WR4': 'WR2', 
    'WR5': 'WR2', 
    'WR6': 'WR2', 
    'WR7': 'WR3', 
    'WR8': 'WR3',
    'TE1':'TE1', 
    'TE2':'TE2', 
    'TE3':'TE3', 
    'TE4':'TE4', 
    'TE5':'TE5', 
    'PK1':'PK1', 
    'PK2':'PK2', 
    'PK3':'PK3',
    'DF1':'DF1'
    })

In [231]:
xl1 = xl1.loc[xl1.season==2021]
xl1 = xl1.reset_index(drop=True)
xl1

,season,week,team,player,pos,posRank,opponent,passA,passC,passY,...,defSack_prior1_opp,defI_prior1_opp,defSaf_prior1_opp,defFum_prior1_opp,defBlk_prior1_opp,defT_prior1_opp,defPtsAgainst_prior1_opp,defPassYAgainst_prior1_opp,defRushYAgainst_prior1_opp,defYdsAgainst_prior1_opp
0,2021.0,1.0,ARI,Kyler Murray,QB,QB1,TEN,32.0,21.0,289.0,...,1.1875,0.9375,0.0000,0.5000,0.0625,0.1250,27.4375,277.4375,120.8125,398.2500
1,2021.0,1.0,KCC,Patrick Mahomes,QB,QB1,CLE,36.0,27.0,337.0,...,2.3750,0.6875,0.1250,0.6250,0.1875,0.1250,26.1875,247.6250,110.8125,358.4375
2,2021.0,1.0,NOS,Jameis Winston,QB,QB1,GBP,20.0,14.0,148.0,...,2.5625,0.6875,0.0625,0.4375,0.0000,0.1250,23.0625,221.1875,112.8125,334.0000
3,2021.0,1.0,TBB,Tom Brady,QB,QB1,DAL,50.0,32.0,379.0,...,1.9375,0.6250,0.0000,0.8125,0.0625,0.1875,29.5625,227.6250,158.8125,386.4375
4,2021.0,1.0,SFO,Jared Goff,QB,QB1,DET,57.0,38.0,338.0,...,1.5000,0.4375,0.0625,0.3125,0.2500,0.0625,32.4375,284.8750,134.8750,419.7500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5760,2021.0,17.0,MIA,Miami DolphinsMIA,DF,DF1,TEN,0.0,0.0,0.0,...,1.1875,0.9375,0.0000,0.5000,0.0625,0.1250,27.4375,277.4375,120.8125,398.2500
5761,2021.0,17.0,DET,Detroit LionsDET,DF,DF1,SEA,0.0,0.0,0.0,...,2.8750,0.8750,0.1875,0.5000,0.1250,0.0000,23.1875,285.0000,95.5625,380.5625
5762,2021.0,17.0,DEN,Denver BroncosDEN,DF,DF1,LAC,0.0,0.0,0.0,...,1.6875,0.7500,0.0625,0.4375,0.1250,0.1250,26.6250,223.6250,119.8125,343.4375
5763,2021.0,17.0,MIN,Minnesota VikingsMIN,DF,DF1,GBP,0.0,0.0,0.0,...,2.5625,0.6875,0.0625,0.4375,0.0000,0.1250,23.0625,221.1875,112.8125,334.0000


In [232]:
# Define features and labels
y = xl1[[
'passA',
'passC',
'passY',
'passT',
'passI',
'pass2',
'rushA',
'rushY',
'rushT',
'rush2',
'recC',
'recY',
'recT',
'rec2',
'fum',
'XPA',
'XPM',
'FGA',
'FGM',
'FG50',
'defSack',
'defI',
'defSaf',
'defFum',
'defBlk',
'defT',
'defPtsAgainst',
'defPassYAgainst',
'defRushYAgainst',
'defYdsAgainst'                                 
]]

X = xl1.drop(columns=[
    'season',
    'team',
    'player',
    'opponent',
    'passA',
    'passC',
    'passY',
    'passT',
    'passI',
    'pass2',
    'rushA',
    'rushY',
    'rushT',
    'rush2',
    'recC',
    'recY',
    'recT',
    'rec2',
    'fum',
    'XPA',
    'XPM',
    'FGA',
    'FGM',
    'FG50',
    'defSack',
    'defI',
    'defSaf',
    'defFum',
    'defBlk',
    'defT',
    'defPtsAgainst',
    'defPassYAgainst',
    'defRushYAgainst',
    'defYdsAgainst'
])

droppedCols = xl1[['season',
'team',
'player',
'pos',
'opponent'
]]

# Encode categorical features
X = pd.get_dummies(X, columns = ['pos', 'posRank'])
X

,week,passA_curr,passC_curr,passY_curr,passT_curr,passI_curr,pass2_curr,rushA_curr,rushY_curr,rushT_curr,...,posRank_RB2,posRank_RB3,posRank_RB4,posRank_RB5,posRank_TE1,posRank_TE2,posRank_TE3,posRank_TE4,posRank_WR1,posRank_WR2
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5760,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
5761,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
5762,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
5763,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [233]:
# Insert any missing feature columns
colsFeatures = ['posRank_DF1', 'posRank_PK1', 'posRank_PK2',
       'posRank_PK3', 'posRank_QB1', 'posRank_QB2', 'posRank_QB3',
       'posRank_QB4', 'posRank_RB1', 'posRank_RB2', 'posRank_RB3',
       'posRank_RB4', 'posRank_RB5',
       'posRank_TE1', 'posRank_TE2', 'posRank_TE3', 'posRank_TE4',
       'posRank_TE5', 'posRank_WR1', 'posRank_WR2', 'posRank_WR3',
       ]
colsToAdd = [x for x in colsFeatures if x not in (X.columns)]
X[colsToAdd] = 0
X

,week,passA_curr,passC_curr,passY_curr,passT_curr,passI_curr,pass2_curr,rushA_curr,rushY_curr,rushT_curr,...,posRank_TE1,posRank_TE2,posRank_TE3,posRank_TE4,posRank_WR1,posRank_WR2,posRank_PK3,posRank_QB4,posRank_TE5,posRank_WR3
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5760,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
5761,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
5762,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
5763,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [234]:
# Put columns in correct order
X = X[['week',
'passA_curr',
'passC_curr',
'passY_curr',
'passT_curr',
'passI_curr',
'pass2_curr',
'rushA_curr',
'rushY_curr',
'rushT_curr',
'rush2_curr',
'recC_curr',
'recY_curr',
'recT_curr',
'rec2_curr',
'fum_curr',
'XPA_curr',
'XPM_curr',
'FGA_curr',
'FGM_curr',
'FG50_curr',
'defSack_curr',
'defI_curr',
'defSaf_curr',
'defFum_curr',
'defBlk_curr',
'defT_curr',
'defPtsAgainst_curr',
'defPassYAgainst_curr',
'defRushYAgainst_curr',
'defYdsAgainst_curr',
'gamesPlayed_curr',
'gamesPlayed_prior1',
'passA_prior1',
'passC_prior1',
'passY_prior1',
'passT_prior1',
'passI_prior1',
'pass2_prior1',
'rushA_prior1',
'rushY_prior1',
'rushT_prior1',
'rush2_prior1',
'recC_prior1',
'recY_prior1',
'recT_prior1',
'rec2_prior1',
'fum_prior1',
'XPA_prior1',
'XPM_prior1',
'FGA_prior1',
'FGM_prior1',
'FG50_prior1',
'defSack_prior1',
'defI_prior1',
'defSaf_prior1',
'defFum_prior1',
'defBlk_prior1',
'defT_prior1',
'defPtsAgainst_prior1',
'defPassYAgainst_prior1',
'defRushYAgainst_prior1',
'defYdsAgainst_prior1',
'gamesPlayed_prior2',
'passA_prior2',
'passC_prior2',
'passY_prior2',
'passT_prior2',
'passI_prior2',
'pass2_prior2',
'rushA_prior2',
'rushY_prior2',
'rushT_prior2',
'rush2_prior2',
'recC_prior2',
'recY_prior2',
'recT_prior2',
'rec2_prior2',
'fum_prior2',
'XPA_prior2',
'XPM_prior2',
'FGA_prior2',
'FGM_prior2',
'FG50_prior2',
'defSack_prior2',
'defI_prior2',
'defSaf_prior2',
'defFum_prior2',
'defBlk_prior2',
'defT_prior2',
'defPtsAgainst_prior2',
'defPassYAgainst_prior2',
'defRushYAgainst_prior2',
'defYdsAgainst_prior2',
'defSack_curr_opp',
'defI_curr_opp',
'defSaf_curr_opp',
'defFum_curr_opp',
'defBlk_curr_opp',
'defT_curr_opp',
'defPtsAgainst_curr_opp',
'defPassYAgainst_curr_opp',
'defRushYAgainst_curr_opp',
'defYdsAgainst_curr_opp',
'defSack_prior1_opp',
'defI_prior1_opp',
'defSaf_prior1_opp',
'defFum_prior1_opp',
'defBlk_prior1_opp',
'defT_prior1_opp',
'defPtsAgainst_prior1_opp',
'defPassYAgainst_prior1_opp',
'defRushYAgainst_prior1_opp',
'defYdsAgainst_prior1_opp',
'pos_DF',
'pos_PK',
'pos_QB',
'pos_RB',
'pos_TE',
'pos_WR',
'posRank_DF1',
'posRank_PK1',
'posRank_PK2',
'posRank_PK3',
'posRank_QB1',
'posRank_QB2',
'posRank_QB3',
'posRank_QB4',
'posRank_RB1',
'posRank_RB2',
'posRank_RB3',
'posRank_RB4',
'posRank_RB5',
'posRank_TE1',
'posRank_TE2',
'posRank_TE3',
'posRank_TE4',
'posRank_TE5',
'posRank_WR1',
'posRank_WR2',
'posRank_WR3',
]]


In [235]:
[print(x) for x in X.columns]

week
passA_curr
passC_curr
passY_curr
passT_curr
passI_curr
pass2_curr
rushA_curr
rushY_curr
rushT_curr
rush2_curr
recC_curr
recY_curr
recT_curr
rec2_curr
fum_curr
XPA_curr
XPM_curr
FGA_curr
FGM_curr
FG50_curr
defSack_curr
defI_curr
defSaf_curr
defFum_curr
defBlk_curr
defT_curr
defPtsAgainst_curr
defPassYAgainst_curr
defRushYAgainst_curr
defYdsAgainst_curr
gamesPlayed_curr
gamesPlayed_prior1
passA_prior1
passC_prior1
passY_prior1
passT_prior1
passI_prior1
pass2_prior1
rushA_prior1
rushY_prior1
rushT_prior1
rush2_prior1
recC_prior1
recY_prior1
recT_prior1
rec2_prior1
fum_prior1
XPA_prior1
XPM_prior1
FGA_prior1
FGM_prior1
FG50_prior1
defSack_prior1
defI_prior1
defSaf_prior1
defFum_prior1
defBlk_prior1
defT_prior1
defPtsAgainst_prior1
defPassYAgainst_prior1
defRushYAgainst_prior1
defYdsAgainst_prior1
gamesPlayed_prior2
passA_prior2
passC_prior2
passY_prior2
passT_prior2
passI_prior2
pass2_prior2
rushA_prior2
rushY_prior2
rushT_prior2
rush2_prior2
recC_prior2
recY_prior2
recT_prior2
rec2_p

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [236]:
y_pred = regressor.predict(X)
y_pred = pd.DataFrame(y_pred)
y_pred.columns = ['passA',
'passC',
'passY',
'passT',
'passI',
'pass2',
'rushA',
'rushY',
'rushT',
'rush2',
'recC',
'recY',
'recT',
'rec2',
'fum',
'XPA',
'XPM',
'FGA',
'FGM',
'FG50',
'defSack',
'defI',
'defSaf',
'defFum',
'defBlk',
'defT',
'defPtsAgainst',
'defPassYAgainst',
'defRushYAgainst',
'defYdsAgainst']
y_pred


,passA,passC,passY,passT,passI,pass2,rushA,rushY,rushT,rush2,...,defSack,defI,defSaf,defFum,defBlk,defT,defPtsAgainst,defPassYAgainst,defRushYAgainst,defYdsAgainst
0,34.432264,22.375579,259.027724,1.709072,0.727895,0.075907,3.768846,17.178561,0.184682,0.008537,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,39.121948,26.105824,312.077151,2.168087,0.740373,0.130234,2.294286,7.206941,0.092182,0.003515,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,34.061944,21.777183,250.879324,1.612406,0.816494,0.077023,2.873848,11.348798,0.113620,0.011293,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,39.873169,26.551786,319.302315,2.231831,0.842024,0.098227,2.077955,6.388642,0.087746,0.003205,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,36.617916,24.134535,278.218561,1.891079,0.718032,0.125194,2.467662,8.925441,0.096781,0.004693,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5760,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.324768,0.856850,0.033445,0.549715,0.078448,0.177573,22.825442,238.567936,109.358979,347.926915
5761,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.021848,0.754777,0.028320,0.482423,0.088326,0.141607,26.494924,241.039353,131.430548,372.469901
5762,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.392767,0.793603,0.031179,0.604393,0.081017,0.183044,21.795754,221.165197,110.355742,331.520940
5763,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.082647,0.827666,0.022963,0.478298,0.102433,0.182791,25.270365,245.863224,125.734313,371.597536


In [203]:
# ['passA',0
# 'passC',0
# 'passY',.04
# 'passT',4
# 'passI',-2
# 'pass2',2
# 'rushA',.1
# 'rushY',.1
# 'rushT',6
# 'rush2',2
# 'recC',.25
# 'recY',.1
# 'recT',6
# 'rec2',2
# 'fum',-2
# 'XPA',0
# 'XPM',1
# 'FGA',0
# 'FGM',3
# 'FG50',5
# 'defSack',1
# 'defI',2
# 'defSaf',2
# 'defFum',2
# 'defBlk',1.5
# 'defT',6
# 'defPtsAgainst',0:10, 6:8, 13:7, 17:5, 21:3, 27:2, 34:0, 45:-1, 59:-3, 99:-5
# 'defPassYAgainst',0
# 'defRushYAgainst',0
# 'defYdsAgainst'] 274:5, 324:2, 375:0, 425:-2, 999:-5

multiplier = [
    0,0,.04,4,-2,2,.1,.1,6,2,.25,.1,6,2,-2,0,1,0,3,5,1,2,2,2,1.5,6,0,0,0,0
]

In [237]:
# Calculate FANTASY scores
multiplier = [
    0,0,.04,4,-2,2,.1,.1,6,2,.25,.1,6,2,-2,0,1,0,3,5,1,2,2,2,1.5,6,0,0,0,0,1,1
]
binList_defPts = [-5,0,6,13,17,21,27,34,45,59,99]
binList_defYds = [0,274,324,375,425,999]

ptList_defPts = [10,8,7,5,3,2,0,-1,-3,-5]
ptList_defYds = [5,2,0,-2,-5]

y['defPtsBin'] = pd.cut(y['defPtsAgainst'], bins=binList_defPts, include_lowest=True, labels=ptList_defPts)
y['defYdsBin'] = pd.cut(y['defYdsAgainst'], bins=binList_defYds, include_lowest=True, labels=ptList_defYds)
y_pred['defPtsBin'] = pd.cut(y_pred['defPtsAgainst'], bins=binList_defPts, include_lowest=True, labels=ptList_defPts)
y_pred['defYdsBin'] = pd.cut(y_pred['defYdsAgainst'], bins=binList_defYds, include_lowest=True, labels=ptList_defYds)

a = droppedCols.merge(y, left_index=True, right_index=True)
a_pred = droppedCols.merge(y_pred, left_index=True, right_index=True)

a.loc[a['pos']!='DF', 'defPtsBin'] = 0
a.loc[a['pos']!='DF', 'defYdsBin'] = 0
a_pred.loc[a_pred['pos']!='DF', 'defPtsBin'] = 0
a_pred.loc[a_pred['pos']!='DF', 'defYdsBin'] = 0

a = a.drop(columns=['season','team','player','pos','opponent'])
a_pred = a_pred.drop(columns=['season','team','player','pos','opponent'])


/Users/travisharry/opt/anaconda3/envs/ffpred/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/travisharry/opt/anaconda3/envs/ffpred/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == "":


In [238]:

def multer(row):
    return row.multiply(multiplier)

b = a.apply(multer, axis=1)
b = b.apply(np.sum, axis=1)
b = pd.DataFrame(b, columns=['real'])

c = a_pred.apply(multer, axis=1)
c = c.apply(np.sum, axis=1)
c = pd.DataFrame(c, columns=['pred'])

In [239]:
real_pred = droppedCols.merge(b, left_index=True, right_index=True).merge(c, left_index=True, right_index=True)
real_pred

,season,team,player,pos,opponent,real,pred
0,2021.0,ARI,Kyler Murray,QB,TEN,34.06,18.708643
1,2021.0,KCC,Patrick Mahomes,QB,CLE,33.78,21.145662
2,2021.0,NOS,Jameis Winston,QB,GBP,30.22,16.730856
3,2021.0,TBB,Tom Brady,QB,DAL,27.16,21.258039
4,2021.0,SFO,Jared Goff,QB,DET,29.22,18.851540
...,...,...,...,...,...,...,...
5760,2021.0,MIA,Miami DolphinsMIA,DF,TEN,3.00,8.387896
5761,2021.0,DET,Detroit LionsDET,DF,SEA,-7.00,7.535018
5762,2021.0,DEN,Denver BroncosDEN,DF,LAC,0.00,8.470904
5763,2021.0,MIN,Minnesota VikingsMIN,DF,GBP,-6.00,7.990896


In [240]:
season_totals = real_pred.groupby(['player', 'pos']).sum()
season_totals.drop(columns=['season'], inplace=True)
season_totals

,,real,pred
player,pos,,
A.J. Brown,WR,120.05,119.515241
A.J. Green,WR,113.00,109.222557
AJ Dillon,RB,170.45,134.288080
Aaron Jones,RB,207.10,179.545450
Aaron Rodgers,QB,321.53,274.975322
...,...,...,...
Zach Pascal,WR,65.25,89.452991
Zach Wilson,QB,144.68,155.117588
Zack Moss,RB,95.15,103.041524


In [241]:
season_totals['diff'] = season_totals.real.sub(season_totals.pred).abs()
season_totals

,,real,pred,diff
player,pos,,,
A.J. Brown,WR,120.05,119.515241,0.534759
A.J. Green,WR,113.00,109.222557,3.777443
AJ Dillon,RB,170.45,134.288080,36.161920
Aaron Jones,RB,207.10,179.545450,27.554550
Aaron Rodgers,QB,321.53,274.975322,46.554678
...,...,...,...,...
Zach Pascal,WR,65.25,89.452991,24.202991
Zach Wilson,QB,144.68,155.117588,10.437588
Zack Moss,RB,95.15,103.041524,7.891524


In [242]:
season_totals.sort_values(by='diff', ascending=False).head(20)

,,real,pred,diff
player,pos,,,
Cooper Kupp,WR,309.70,230.358002,79.341998
Josh Allen,QB,390.42,311.819954,78.600046
Dallas CowboysDAL,DF,212.00,134.858601,77.141399
Jonathan Taylor,RB,364.55,293.220592,71.329408
Ja'Marr Chase,WR,241.45,173.195919,68.254081
James Conner,RB,225.85,158.668540,67.181460
Jalen Hurts,QB,326.06,260.566955,65.493045
Deebo Samuel,WR,260.35,199.776245,60.573755
Jacksonville JaguarsJAX,DF,67.00,125.442182,58.442182


In [243]:
season_totals.sort_values(by='real', ascending=False).head(20)

,,real,pred,diff
player,pos,,,
Josh Allen,QB,390.42,311.819954,78.600046
Jonathan Taylor,RB,364.55,293.220592,71.329408
Justin Herbert,QB,359.44,309.889820,49.550180
Tom Brady,QB,352.50,314.333765,38.166235
Patrick Mahomes,QB,341.16,312.996246,28.163754
Jalen Hurts,QB,326.06,260.566955,65.493045
Aaron Rodgers,QB,321.53,274.975322,46.554678
Joe Burrow,QB,318.24,313.683277,4.556723
Matthew Stafford,QB,315.42,270.519925,44.900075


In [244]:
season_totals.sort_values(by='pred', ascending=False).head(20)

,,real,pred,diff
player,pos,,,
Tom Brady,QB,352.50,314.333765,38.166235
Joe Burrow,QB,318.24,313.683277,4.556723
Patrick Mahomes,QB,341.16,312.996246,28.163754
Josh Allen,QB,390.42,311.819954,78.600046
Justin Herbert,QB,359.44,309.889820,49.550180
Jonathan Taylor,RB,364.55,293.220592,71.329408
Derek Carr,QB,245.52,292.025771,46.505771
Dak Prescott,QB,293.56,285.473681,8.086319
Aaron Rodgers,QB,321.53,274.975322,46.554678


In [ ]:
# Create master player db
# Get 2022 opponents
    # Merge opponents to coming stats
# Get ages
    # Scrape ages
    # Connect ages to historicalstats
# Deal with currentstats throwing things out of whack before season starts
# Add in Punt- and Kick-Return scores